In [2]:
import torch
from torch import nn
from torchvision.datasets import MNIST # training dataset
import matplotlib.pyplot as plt
torch.manual_seed(0)


In [3]:
import numpy as np

In [4]:
# not sure
from torchvision import transforms
from torchvision.utils import make_grid
from torch.utils.data import DataLoader

In [4]:
# totally no idea
from tqdm import tqdm

In [5]:
def show_tensor_imgaes(image_tensor, num_images, size):
    '''
    Function for visualizing images: Given a tensor of images, number of images, and
    size per image, plots and prints the images in a uniform grid.

    '''
    # not sure
    image_unflat = image_tensor.detach().cpu().view(-1, *size)
    image_grid = make_grid(image_unflat[:num_images], nrow=5)
    # not sure
    plt.imshow(image_grid.permute(1, 2, 0).squeeze())
    plt.show()

In [6]:
def get_generator_block(input_dim, output_dim):
    """
    """
    return nn.Sequential(
        nn.Linear(input_dim, output_dim),
        nn.BatchNorm1d(output_dim),
        nn.ReLU(inplace=True)
    )
## 大概知道什么意思但是细节不清楚

In [7]:
input = torch.randn(50,30)
print ("input size is :" + str(input.size()))
m = nn.Linear(30, 20)
output = m(input)
print ("output size is :" + str(output.size()))
# mm, same as FNN

input size is :torch.Size([50, 30])
output size is :torch.Size([50, 20])


In [8]:
m = nn.BatchNorm1d(20)
output2 = m(output)
output2.size()
# don't know what it for, but the it should same as the output column number

torch.Size([50, 20])

In [9]:
class Generator(nn.Module):
    ## initialize the neural network
    def __init__ (self, z_dim = 10, im_dim = 784, hidden_dim = 128):
        super(Generator, self).__init__()
        self.gen = nn.Sequential(
            get_generator_block(z_dim, hidden_dim),
            # 10 * 128
            get_generator_block(hidden_dim, hidden_dim * 2),
            # 128 * 256
            get_generator_block(hidden_dim * 2, hidden_dim * 4),
            # 256 * 512
            get_generator_block(hidden_dim * 4, hidden_dim * 8),
            # 512 * 1024??
            nn.Linear(hidden_dim * 8, im_dim),
            nn.Sigmoid()
        )

    def forward(self, noise):
        return self.gen(noise)

    def get_gen(self):
        return self.gen
        

In [10]:
def get_noise(n_samples, z_dim, device= 'cpu'):
    return torch.randn(n_samples, z_dim, device = device)


In [13]:
def get_discriminator_block(input_dim, output_dim):
    return nn.Sequential(
        nn.Linear(input_dim, output_dim),
        nn.LeakyReLU(0.2, inplace = True)
    )

In [27]:
class Discriminator(nn.Module):

    def __init__ (self, im_dim = 784, hidden_dim=128):
        super(Discriminator, self).__init__()
        self.disc = nn.Sequential(
            get_discriminator_block(im_dim, hidden_dim * 4),
            get_discriminator_block(hidden_dim * 4, hidden_dim * 2),
            get_discriminator_block(hidden_dim * 2, hidden_dim),
            nn.Linear(hidden_dim,1),
    )

    def forward(self, image):
        return self.disc(image)
    
    def get_disc(self):
        return self.disc

Training

In [24]:
criterion = nn.BCEWithLogitsLoss()
n_epochs = 200
z_dim = 64
# z dimension is the dimension of noise
display_step = 500
batch_size = 128
lr = 0.00001

In [22]:
dataloader = DataLoader(
    MNIST('.', download=True, transform=transforms.ToTensor()),
    batch_size=batch_size,
    shuffle=True)


0it [00:00, ?it/s]Downloading http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz to ./MNIST/raw/train-images-idx3-ubyte.gz


HTTPError: HTTP Error 503: Service Unavailable

In [23]:
device = 'cpu'

In [25]:
gen = Generator(z_dim).to(device)
gen_opt = torch.optim.Adam(gen.parameters(), lr = lr)

In [28]:
disc = Discriminator().to(device)
disc_opt = torch.optim.Adam(disc.parameters(), lr = lr)

In [ ]:
def get_disc_loss(gen, disc, criterion, real, num_images, z_dim, device):
    